Build linear regression model using lasso , use parameter tuning to find appropriate value for hyper parameters . check how many coefficients are made zero.
Build linear regression model using ridge , use parameter tuning to find appropriate value for hyper parameters
Linear regression outcome, theoretically can take negative values as well, how can you ensure that your outcomes are positive

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_file=r'C:\Users\Nikhil\Desktop\AI\Data\Data\facebook_comments.csv'

In [3]:
df=pd.read_csv(data_file)

In [4]:
df.shape

(40949, 42)

In [5]:
df.head()

,likes,checkins,talking_about,page_category,feat5,feat6,feat7,feat8,feat9,feat10,...,CC4,CC5,Base_Time,Post_Length,Post Share Count,Post Promotion Status,H local,Comments_in_next_H_hrs,Post Published Weekday,Base Date Time Weekday
0,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,65,166,2,0,24,0,Wednesday,Saturday
1,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,10,132,1,0,24,0,Thursday,Friday
2,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,14,133,2,0,24,0,Friday,Saturday
3,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,7,-3,62,131,1,0,24,0,Friday,Monday
4,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,58,142,5,0,24,0,Monday,Wednesday


In [6]:
k=df['page_category'].value_counts()

In [7]:
cats=k[k>200].index[:-1]

In [8]:
for cat in cats:
    name='page_category_'+cat
    df[cat]=(df['page_category']==cat).astype(int)
del df['page_category']

In [9]:
df.shape

(40949, 75)

In [10]:
df['Post Published Weekday'].unique()

array(['Wednesday', 'Thursday', 'Friday', 'Monday', 'Tuesday', 'Sunday',
       'Saturday'], dtype=object)

In [11]:
cols=['Post Published Weekday','Base Date Time Weekday']

In [12]:
dic={'Wednesday':4, 'Thursday':5, 'Friday':6, 'Monday':2, 'Tuesday':3, 'Sunday':1,
       'Saturday':7}

In [13]:
for col in cols:
    df[col]=df[col].map(dic)

In [14]:
for col in cols:
    df[col+'_sin']=np.sin(2*np.pi*df[col]/7)
    df[col+'_cos']=np.cos(2*np.pi*df[col]/7)
    del df[col]

In [15]:
target='Comments_in_next_H_hrs'

In [16]:
x_train=df.drop(target,1)
y_train=df[target]

In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

In [18]:
model=LinearRegression()

In [19]:
scores=cross_val_score(model,x_train,y_train,cv=10,n_jobs=-1,
                       scoring='neg_mean_absolute_error',verbose=20)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done   2 out of  10 | elapsed:    3.7s remaining:   14.8s
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    3.7s remaining:    8.7s
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    3.7s remaining:    5.6s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    3.7s remaining:    3.7s
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    3.7s remaining:    2.5s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    3.7s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:    3.7s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    3.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    3.7s finished


In [20]:
simple_lm_error=abs(scores.mean())
simple_lm_error

8.48670468677086

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge,Lasso

In [22]:
model_lasso=Lasso(fit_intercept=True)

In [23]:
alpha=np.linspace(1,100,10)

In [24]:
params={'alpha':alpha}

In [25]:
grid_search=GridSearchCV(model_lasso,cv=10,scoring='neg_mean_absolute_error',
                         verbose=20,n_jobs=-1,param_grid=params)

In [26]:
grid_search.fit(x_train,y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   21.1s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   26.7s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   28.2s
[Paralle

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': array([  1.,  12.,  23.,  34.,  45.,  56.,  67.,  78.,  89., 100.])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=20)

In [27]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
# you should also try this 
# source : https://www.kaggle.com/grfiv4/displaying-the-results-of-a-grid-search

In [28]:
report(grid_search.cv_results_,5) # you can do tuning on a finer range around 78

Model with rank: 1
Mean validation score: -6.948 (std: 1.468)
Parameters: {'alpha': 78.0}

Model with rank: 2
Mean validation score: -6.977 (std: 1.486)
Parameters: {'alpha': 89.0}

Model with rank: 3
Mean validation score: -6.996 (std: 1.466)
Parameters: {'alpha': 100.0}

Model with rank: 4
Mean validation score: -7.014 (std: 1.422)
Parameters: {'alpha': 67.0}

Model with rank: 5
Mean validation score: -7.153 (std: 1.380)
Parameters: {'alpha': 56.0}



In [29]:
lasso=grid_search.best_estimator_

In [30]:
lasso.fit(x_train,y_train)

Lasso(alpha=78.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [31]:
(lasso.coef_==0).sum()

67

In [32]:
# list(zip(x_train.columns,lasso.coef_))
# for looking at the coefficients 

In [33]:
model_ridge=Ridge(fit_intercept=True)
alpha=np.linspace(10000,100000,10)
# we arrived at this after many trials and it still seems that more penalty will be better 
# you can try expanding this further and see if this improves the ridge performance
params={'alpha':alpha}

In [34]:
grid_search=GridSearchCV(model_ridge,cv=10,scoring='neg_mean_absolute_error',
                         verbose=20,n_jobs=-1,param_grid=params)

In [35]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': array([ 10000.,  20000.,  30000.,  40000.,  50000.,  60000.,  70000.,
        80000.,  90000., 100000.])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=20)

In [36]:
report(grid_search.cv_results_,5)

Model with rank: 1
Mean validation score: -8.351 (std: 1.103)
Parameters: {'alpha': 100000.0}

Model with rank: 2
Mean validation score: -8.353 (std: 1.102)
Parameters: {'alpha': 90000.0}

Model with rank: 3
Mean validation score: -8.356 (std: 1.101)
Parameters: {'alpha': 80000.0}

Model with rank: 4
Mean validation score: -8.360 (std: 1.100)
Parameters: {'alpha': 70000.0}

Model with rank: 5
Mean validation score: -8.363 (std: 1.100)
Parameters: {'alpha': 60000.0}



In [37]:
# so far the performance of lasso seems to be the best cv MAE being 6.948

In [38]:
## positive predictions only , use log(target) as the target and reverse transform 
# for getting original target predictions

In [39]:
y_train=np.log(y_train+100) # for avoiding log(0)

In [40]:
model_lasso=Lasso(fit_intercept=True)

In [41]:
alpha=np.linspace(0,10,10)

In [42]:
params={'alpha':alpha}

In [43]:
grid_search=GridSearchCV(model_lasso,cv=10,scoring='neg_mean_absolute_error',
                         verbose=20,n_jobs=-1,param_grid=params)

In [44]:
grid_search.fit(x_train,y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    8.7s
[Paralle

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': array([ 0.     ,  1.11111,  2.22222,  3.33333,  4.44444,  5.55556,
        6.66667,  7.77778,  8.88889, 10.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=20)

In [45]:
report(grid_search.cv_results_,5) # error value here is not equivalent to what we were seeing earlier 

Model with rank: 1
Mean validation score: -0.047 (std: 0.007)
Parameters: {'alpha': 1.1111111111111112}

Model with rank: 2
Mean validation score: -0.049 (std: 0.007)
Parameters: {'alpha': 2.2222222222222223}

Model with rank: 3
Mean validation score: -0.050 (std: 0.005)
Parameters: {'alpha': 0.0}

Model with rank: 4
Mean validation score: -0.053 (std: 0.008)
Parameters: {'alpha': 3.3333333333333335}

Model with rank: 5
Mean validation score: -0.057 (std: 0.008)
Parameters: {'alpha': 4.444444444444445}



In [46]:
original_predictions=np.exp(grid_search.predict(x_train))-100

In [47]:
(original_predictions<0).sum() # there are still few negative preds becasue of -100
# in practice you can actually set them to be zero 

43

In [48]:
np.mean(abs(df[target]-original_predictions))

6.685280621766945

In [49]:
np.mean(abs(df[target]-np.where(original_predictions<0,0,original_predictions)))

6.685186760258289